# Explicit regularization

In [ ]:
import pandas as pd

def read_csv(filename):
    return pd.read_csv("../thesis_data/{}".format(filename), index_col=0)

In [ ]:
measure_names = [
    "gradient_norm_test",
    "path_length_f_test",
    "path_length_d_test",
    "weights_product",
]

def get_ratios(
    ms: pd.DataFrame, base_mask: pd.DataFrame, normed_col: str, match_col="dataset.name"
):
    """
    Computes normalized values of a dataframe's column by dividing by the value
    in a "corresponding" row. Used e.g. when explicitly regularizing smoothness measures
    """
    ms = ms.copy()
    base = ms[base_mask]
    assert base[match_col].is_unique

    normed_col_after = normed_col + "_normalized"

    # Inefficient, but good enough
    for _, row in base.iterrows():
        cur = ms.loc[ms[match_col] == row[match_col]]
        ms.loc[ms[match_col] == row[match_col], normed_col_after] = (
            cur[normed_col] / row[normed_col]
        )

    return ms

In [ ]:
summary = pd.DataFrame(
    index=measure_names,
    columns=["Unregularized mean", "Regularized mean", "Lower bound", "Normalized mean", "Normalized std"],
    dtype=float,
)

summary["Lower bound"] = [0., 1., 0., 0.]

## Gradient norm

In [ ]:
measures = read_csv("explicit_reg_gradient_norm.csv")
measures = measures.loc[measures["model.learning_rate"] == 0.01]
measures = get_ratios(measures, measures["model.gradient_norm_reg_coef"] == 0, "gradient_norm_test")

def update_summary(measures, measure_name, reg_coef_col_name, baseline=0):
    if baseline == 0:
        measure_name_normalized = measure_name + "_normalized"
    else:
        measure_name_normalized = measure_name + "_baselined_normalized"
    
    unreg_mean = measures[measures[reg_coef_col_name] == 0][measure_name].mean()
    summary.loc[measure_name, "Unregularized mean"] = unreg_mean

    largest_coef = max(measures[reg_coef_col_name])
    stats = measures[measures[reg_coef_col_name] == largest_coef].describe()

    summary.loc[measure_name, "Regularized mean"] = stats.loc["mean", measure_name]
    summary.loc[measure_name, "Normalized mean"] = stats.loc["mean", measure_name_normalized]
    summary.loc[measure_name, "Normalized std"] = stats.loc["std", measure_name_normalized]

update_summary(measures, "gradient_norm_test", "model.gradient_norm_reg_coef")

## Weights product

In [ ]:
measures = read_csv("explicit_reg_weights_product.csv")
measures = measures.loc[measures["model.learning_rate"] == 0.01]

# Only take into account the models which converged in the regularized version.
measures = measures.loc[measures["actual_epochs"] < 25000]
measures = measures[measures["dataset.name"].isin(
    set(measures.loc[measures["model.weights_product_reg_coef"].round(10) == 3e-5, "dataset.name"])
)]

measures = get_ratios(measures, measures["model.weights_product_reg_coef"] == 0, "weights_product")

update_summary(measures, "weights_product", "model.weights_product_reg_coef")

## Function path length

In [ ]:
measures = read_csv("../thesis_data/explicit_reg_function_path_length.csv")

# Only take into account the models which converged in the regularized version.
measures = measures.loc[measures["actual_epochs"] < 25000]
measures = measures[measures["dataset.name"].isin(
    set(measures.loc[measures["model.path_length_f_reg_coef"].round(10) == 0.0001, "dataset.name"])
)]

measures["path_length_f_test_baselined"] = measures["path_length_f_test"] - 1
measures = get_ratios(measures, measures["model.path_length_f_reg_coef"] == 0, "path_length_f_test_baselined")

update_summary(measures, "path_length_f_test", "model.path_length_f_reg_coef", baseline=1)

## Gradient path length

In [ ]:
measures = read_csv("../thesis_data/explicit_reg_gradient_path_length.csv")

# Only take into account the models which converged in the regularized version.
measures = measures.loc[measures["actual_epochs"] < 25000]
measures = measures[measures["dataset.name"].isin(
    set(measures.loc[measures["model.path_length_d_reg_coef"].round(10) == 0.00001, "dataset.name"])
)]

measures = get_ratios(measures, measures["model.path_length_d_reg_coef"] == 0, "path_length_d_test")

update_summary(measures, "path_length_d_test", "model.path_length_d_reg_coef")

## Results

In [ ]:
summary.round(2)